<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/PyPageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pyspark

In [1]:
#!pip install -q findspark
import findspark
findspark.init()

# SPARK INSTALLED... lets play

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/15 09:27:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#!wget -q https://storage.googleapis.com/public_lddm_data/small_page_links.nt
!ls

CharSets.ipynb			  map_reduce_with_py_sparl.ipynb
GraphFramesPageRank.ipynb	  page_links_en.nt.bz2
PageRank_an_PySPARK_SQL.ipynb	  people.csv
PyPageRank.ipynb		  pyspark
README.md			  run.sh
Spark_Structured_Streaming.ipynb  small_page_links.nt
Streaming_CSV.ipynb		  stream.py
VoiD_with_PySpark_RDD.ipynb	  testPyPageRank.py
Wikipedia_Events.ipynb		  vianu.pig
authors.pig			  watdiv-100k.nt
data				  webdam-books.txt
dataproc.py			  webdam-publishers.txt
exam2022.ipynb			  wordcount.sh
flatten2.pig


In [3]:
lines = spark.read.text("gs://beshoux-large_bucket/small_page_links.nt").rdd.map(lambda r: r[0])
lines.take(5)

24/11/15 09:27:33 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://beshoux-large_bucket/small_page_links.nt.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameRead

Py4JJavaError: An error occurred while calling o29.text.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(Dat

In [6]:
import re
def computeContribs(urls, rank) :
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[2]

In [7]:
# Loads all URLs from input file and initialize their neighbors.
links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()

# Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [8]:
links.take(5)

24/11/06 15:59:21 WARN BlockManager: Task 15 already completed, not releasing lock for rdd_14_0


[('<http://dbpedia.org/resource/List_of_countries_and_outlying_territories_by_total_area>',
  <pyspark.resultiterable.ResultIterable at 0x7d6868215f40>),
 ('<http://dbpedia.org/resource/Gospel_of_Peter>',
  <pyspark.resultiterable.ResultIterable at 0x7d68844057c0>),
 ('<http://dbpedia.org/resource/Wells%2C_Maine>',
  <pyspark.resultiterable.ResultIterable at 0x7d68680a9e80>),
 ('<http://dbpedia.org/resource/Greenville%2C_Florida>',
  <pyspark.resultiterable.ResultIterable at 0x7d68680a9d60>),
 ('<http://dbpedia.org/resource/William_Hamilton_%28Jacobite_poet%29>',
  <pyspark.resultiterable.ResultIterable at 0x7d68680a9d00>)]

In [9]:
#groupByKey makes lists !!
links.map(lambda x: (x[0],list(x[1]))).take(5)

24/11/06 15:59:23 WARN BlockManager: Task 16 already completed, not releasing lock for rdd_14_0


[('<http://dbpedia.org/resource/List_of_countries_and_outlying_territories_by_total_area>',
  ['<http://dbpedia.org/resource/Niger>',
   '<http://dbpedia.org/resource/Agalega_Islands>',
   '<http://dbpedia.org/resource/List_of_island_countries_by_area>',
   '<http://dbpedia.org/resource/Panama>',
   '<http://dbpedia.org/resource/Gal%C3%A1pagos_Islands>',
   '<http://dbpedia.org/resource/%C3%85land_Islands>',
   '<http://dbpedia.org/resource/Isla_del_Coco>',
   '<http://dbpedia.org/resource/Bosnia_and_Herzegovina>',
   '<http://dbpedia.org/resource/Burkina_Faso>',
   '<http://dbpedia.org/resource/Lithuania>',
   '<http://dbpedia.org/resource/Maldives>',
   '<http://dbpedia.org/resource/Tonga>',
   '<http://dbpedia.org/resource/Samoa>',
   '<http://dbpedia.org/resource/Auckland_Islands>',
   '<http://dbpedia.org/resource/Afghanistan>',
   '<http://dbpedia.org/resource/Slovenia>',
   '<http://dbpedia.org/resource/Overseas_collectivity>',
   '<http://dbpedia.org/resource/Peru>',
   '<http:

In [10]:
#groupByKey makes lists !!


24/11/06 16:00:23 WARN MemoryStore: Not enough space to cache rdd_14_1 in memory! (computed 131.7 MiB so far)
24/11/06 16:00:23 WARN BlockManager: Block rdd_14_1 could not be removed as it was not found on disk or in memory
24/11/06 16:00:23 WARN BlockManager: Putting block rdd_14_1 failed
24/11/06 16:01:30 WARN MemoryStore: Not enough space to cache rdd_14_2 in memory! (computed 132.3 MiB so far)
24/11/06 16:01:30 WARN BlockManager: Block rdd_14_2 could not be removed as it was not found on disk or in memory
24/11/06 16:01:30 WARN BlockManager: Putting block rdd_14_2 failed
24/11/06 16:02:36 WARN MemoryStore: Not enough space to cache rdd_14_3 in memory! (computed 127.3 MiB so far)
24/11/06 16:02:36 WARN BlockManager: Block rdd_14_3 could not be removed as it was not found on disk or in memory
24/11/06 16:02:36 WARN BlockManager: Putting block rdd_14_3 failed
24/11/06 16:03:48 WARN MemoryStore: Not enough space to cache rdd_14_4 in memory! (computed 178.6 MiB so far)
24/11/06 16:03:48

In [ ]:
ranks.take(5)

In [ ]:
links.join(ranks).take(5)

In [ ]:
links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(
            url_urls_rank[1][0], url_urls_rank[1][1]  # type: ignore[arg-type]
        )).take(5)

In [ ]:
from operator import add
for iteration in range(1):
  # Calculates URL contributions to the rank of other URLs.
  contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(
            url_urls_rank[1][0], url_urls_rank[1][1]  # type: ignore[arg-type]
        ))

  # Re-calculates URL ranks based on neighbor contributions.
  ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

    # Collects all URL ranks and dump them to console.
for (link, rank) in ranks.collect():
  print("%s has rank: %s." % (link, rank))


# --- Implémentation avec PySpark RDD ---

In [ ]:
# Fonction pour parser les voisins (identique à celle utilisée dans DataFrame)
def parseNeighborsRDD(urls):
    parts = re.split(r'\s+', urls)
    return parts[0], parts[2]

# Fonction pour calculer les contributions de rang (identique à celle utilisée dans DataFrame)
def computeContribsRDD(urls, rank):
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

In [ ]:
# Chargement des données et initialisation avec RDD
lines_rdd = spark.read.text("page_links_en.nt.bz2").rdd.map(lambda r: r[0])  # Remplace par ton chemin

# Initialisation des liens et des rangs avec RDD
links_rdd = lines_rdd.map(lambda urls: parseNeighborsRDD(urls)).distinct().groupByKey().cache()
ranks_rdd = links_rdd.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [ ]:
# Boucle d'itérations pour le calcul de PageRank avec RDD
num_iterations = 10
for iteration in range(num_iterations):
    contribs_rdd = links_rdd.join(ranks_rdd).flatMap(lambda url_urls_rank: computeContribsRDD(
        url_urls_rank[1][0], url_urls_rank[1][1]
    ))
    
    ranks_rdd = contribs_rdd.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

In [ ]:
print("Résultats PageRank avec RDD:")
for (link, rank) in ranks_rdd.collect():
    print("%s has rank: %s." % (link, rank))